## From CSV

In [2]:
import pandas as pd
import os
os.makedirs("data/structuredfiles",exist_ok=True)

In [4]:
# Create sample data
data = {
    'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'],
    'Category': ['Electronics', 'Accessories', 'Accessories', 'Electronics', 'Electronics'],
    'Price': [999.99, 29.99, 79.99, 299.99, 89.99],
    'Stock': [50, 200, 150, 75, 100],
    'Description': [
        'High-performance laptop with 16GB RAM and 512GB SSD',
        'Wireless optical mouse with ergonomic design',
        'Mechanical keyboard with RGB backlighting',
        '27-inch 4K monitor with HDR support',
        '1080p webcam with noise cancellation'
    ]
}

# Save as CSV
df = pd.DataFrame(data)
df.to_csv('data/structuredfiles/products.csv', index=False)

In [6]:
# Save as Excel with multiple sheets
with pd.ExcelWriter('data/structuredfiles/inventory.xlsx') as writer:
    df.to_excel(writer, sheet_name='Products', index=False)
    
    # Add another sheet
    summary_data = {
        'Category': ['Electronics', 'Accessories'],
        'Total_Items': [3, 2],
        'Total_Value': [1389.97, 109.98]
    }
    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)

CSV Processing

In [7]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.document_loaders import UnstructuredCSVLoader

/Users/maruthienugula/RAG_learnings/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Method 1: CSVLoader - Each row becomes a document
print(" CSVLoader - Row-based Documents")
csv_loader = CSVLoader(
    file_path='data/structuredfiles/products.csv',
    encoding='utf-8',
    csv_args={
        'delimiter': ',',
        'quotechar': '"',
    }
)

csv_docs = csv_loader.load()
print(csv_docs)
print(f"Loaded {len(csv_docs)} documents (one per row)")
print("\nFirst document:")
print(f"Content: {csv_docs[0].page_content}")
print(f"Metadata: {csv_docs[0].metadata}")

 CSVLoader - Row-based Documents
[Document(metadata={'source': 'data/structuredfiles/products.csv', 'row': 0}, page_content='Product: Laptop\nCategory: Electronics\nPrice: 999.99\nStock: 50\nDescription: High-performance laptop with 16GB RAM and 512GB SSD'), Document(metadata={'source': 'data/structuredfiles/products.csv', 'row': 1}, page_content='Product: Mouse\nCategory: Accessories\nPrice: 29.99\nStock: 200\nDescription: Wireless optical mouse with ergonomic design'), Document(metadata={'source': 'data/structuredfiles/products.csv', 'row': 2}, page_content='Product: Keyboard\nCategory: Accessories\nPrice: 79.99\nStock: 150\nDescription: Mechanical keyboard with RGB backlighting'), Document(metadata={'source': 'data/structuredfiles/products.csv', 'row': 3}, page_content='Product: Monitor\nCategory: Electronics\nPrice: 299.99\nStock: 75\nDescription: 27-inch 4K monitor with HDR support'), Document(metadata={'source': 'data/structuredfiles/products.csv', 'row': 4}, page_content='Produc

In [10]:
csv_docs

[Document(metadata={'source': 'data/structuredfiles/products.csv', 'row': 0}, page_content='Product: Laptop\nCategory: Electronics\nPrice: 999.99\nStock: 50\nDescription: High-performance laptop with 16GB RAM and 512GB SSD'),
 Document(metadata={'source': 'data/structuredfiles/products.csv', 'row': 1}, page_content='Product: Mouse\nCategory: Accessories\nPrice: 29.99\nStock: 200\nDescription: Wireless optical mouse with ergonomic design'),
 Document(metadata={'source': 'data/structuredfiles/products.csv', 'row': 2}, page_content='Product: Keyboard\nCategory: Accessories\nPrice: 79.99\nStock: 150\nDescription: Mechanical keyboard with RGB backlighting'),
 Document(metadata={'source': 'data/structuredfiles/products.csv', 'row': 3}, page_content='Product: Monitor\nCategory: Electronics\nPrice: 299.99\nStock: 75\nDescription: 27-inch 4K monitor with HDR support'),
 Document(metadata={'source': 'data/structuredfiles/products.csv', 'row': 4}, page_content='Product: Webcam\nCategory: Electron

In [11]:
from typing import List
from langchain_core.documents import Document

# Method 2: Custom CSV processing
# This is easy and understandable
print("Custom CSV Processing")
def process_csv_intelligently(filepath: str) -> List[Document]:
    """Process CSV with intelligent document creation"""
    df = pd.read_csv(filepath)
    documents = []
    
    # Strategy 1: One document per row with structured content
    for idx, row in df.iterrows():
        # Create structured content
        content = f"""Product Information:
        Name: {row['Product']}
        Category: {row['Category']}
        Price: ${row['Price']}
        Stock: {row['Stock']} units
        Description: {row['Description']}"""
        
        # Create document with rich metadata
        doc = Document(
            page_content=content,
            metadata={
                'source': filepath,
                'row_index': idx,
                'product_name': row['Product'],
                'category': row['Category'],
                'price': row['Price'],
                'data_type': 'product_info'
            }
        )
        documents.append(doc)
    return documents

Custom CSV Processing


In [12]:
process_csv_intelligently('data/structuredfiles/products.csv')

[Document(metadata={'source': 'data/structuredfiles/products.csv', 'row_index': 0, 'product_name': 'Laptop', 'category': 'Electronics', 'price': 999.99, 'data_type': 'product_info'}, page_content='Product Information:\n        Name: Laptop\n        Category: Electronics\n        Price: $999.99\n        Stock: 50 units\n        Description: High-performance laptop with 16GB RAM and 512GB SSD'),
 Document(metadata={'source': 'data/structuredfiles/products.csv', 'row_index': 1, 'product_name': 'Mouse', 'category': 'Accessories', 'price': 29.99, 'data_type': 'product_info'}, page_content='Product Information:\n        Name: Mouse\n        Category: Accessories\n        Price: $29.99\n        Stock: 200 units\n        Description: Wireless optical mouse with ergonomic design'),
 Document(metadata={'source': 'data/structuredfiles/products.csv', 'row_index': 2, 'product_name': 'Keyboard', 'category': 'Accessories', 'price': 79.99, 'data_type': 'product_info'}, page_content='Product Informatio

Excel Sheet Processing

In [13]:
# Method 1: Using pandas for full control
print("Pandas-based Excel Processing")
def process_excel_with_pandas(filepath: str) -> List[Document]:
    """Process Excel with sheet awareness"""
    documents = []
    
    # Read all sheets
    excel_file = pd.ExcelFile(filepath)
    
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel(filepath, sheet_name=sheet_name)
        
        # Create document for each sheet
        sheet_content = f"Sheet: {sheet_name}\n"
        sheet_content += f"Columns: {', '.join(df.columns)}\n"
        sheet_content += f"Rows: {len(df)}\n\n"
        sheet_content += df.to_string(index=False)
        
        doc = Document(
            page_content=sheet_content,
            metadata={
                'source': filepath,
                'sheet_name': sheet_name,
                'num_rows': len(df),
                'num_columns': len(df.columns),
                'data_type': 'excel_sheet'
            }
        )
        documents.append(doc)
    
    return documents

Pandas-based Excel Processing


In [15]:
excel_docs = process_excel_with_pandas('data/structuredfiles/inventory.xlsx')
print(f"Processed {len(excel_docs)} sheets")
print(excel_docs)

Processed 2 sheets
[Document(metadata={'source': 'data/structuredfiles/inventory.xlsx', 'sheet_name': 'Products', 'num_rows': 5, 'num_columns': 5, 'data_type': 'excel_sheet'}, page_content='Sheet: Products\nColumns: Product, Category, Price, Stock, Description\nRows: 5\n\n Product    Category  Price  Stock                                         Description\n  Laptop Electronics 999.99     50 High-performance laptop with 16GB RAM and 512GB SSD\n   Mouse Accessories  29.99    200        Wireless optical mouse with ergonomic design\nKeyboard Accessories  79.99    150           Mechanical keyboard with RGB backlighting\n Monitor Electronics 299.99     75                 27-inch 4K monitor with HDR support\n  Webcam Electronics  89.99    100                1080p webcam with noise cancellation'), Document(metadata={'source': 'data/structuredfiles/inventory.xlsx', 'sheet_name': 'Summary', 'num_rows': 2, 'num_columns': 3, 'data_type': 'excel_sheet'}, page_content='Sheet: Summary\nColumns: Cat

UnstructuredExcelLoader from langchain_community.document_loaders

## Semantic Meaning is kept intact


In [24]:
from langchain_community.document_loaders import UnstructuredExcelLoader
# Method 2: UnstructuredExcelLoader
print("UnstructuredExcelLoader")
try:
    excel_loader = UnstructuredExcelLoader(
        'data/structured_files/inventory.xlsx',
        mode="elements"
    )
    unstructured_docs = excel_loader.load()
    print(" Handles complex Excel features")
    print(" Preserves formatting info")
    print("Requires unstructured library")
except Exception as e:
    print("Requires unstructured library with Excel support")

UnstructuredExcelLoader
Requires unstructured library with Excel support


In [25]:
unstructured_docs

NameError: name 'unstructured_docs' is not defined

This is because unstructuredexcelloader doesn't work even after unstructuredexcelloader[all-docs] is installed

In [27]:
from unstructured.partition.xlsx import partition_xlsx
from langchain_core.documents import Document

print("Manual Unstructured Excel Pipeline")

elements = partition_xlsx("data/structuredfiles/inventory.xlsx")

docs = []
for el in elements:
    if el.text and el.text.strip():
        docs.append(
            Document(
                page_content=el.text,
                metadata={
                    **(el.metadata.to_dict() if el.metadata else {}),
                    "source": "inventory.xlsx",
                    "loader": "unstructured_partition_xlsx"
                }
            )
        )

print(" Handles complex Excel features")
print(" Preserves formatting info")
print(" Uses unstructured Excel backend directly")
print(f" Loaded {len(docs)} documents")


Manual Unstructured Excel Pipeline
 Handles complex Excel features
 Preserves formatting info
 Uses unstructured Excel backend directly
 Loaded 2 documents


In [28]:
docs

[Document(metadata={'file_directory': 'data/structuredfiles', 'filename': 'inventory.xlsx', 'last_modified': '2026-01-23T22:56:41', 'page_name': 'Products', 'page_number': 1, 'text_as_html': '<table><tr><td>Product</td><td>Category</td><td>Price</td><td>Stock</td><td>Description</td></tr><tr><td>Laptop</td><td>Electronics</td><td>999.99</td><td>50</td><td>High-performance laptop with 16GB RAM and 512GB SSD</td></tr><tr><td>Mouse</td><td>Accessories</td><td>29.99</td><td>200</td><td>Wireless optical mouse with ergonomic design</td></tr><tr><td>Keyboard</td><td>Accessories</td><td>79.99</td><td>150</td><td>Mechanical keyboard with RGB backlighting</td></tr><tr><td>Monitor</td><td>Electronics</td><td>299.99</td><td>75</td><td>27-inch 4K monitor with HDR support</td></tr><tr><td>Webcam</td><td>Electronics</td><td>89.99</td><td>100</td><td>1080p webcam with noise cancellation</td></tr></table>', 'languages': ['eng'], 'filetype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.s